In [11]:
import torch
from torch.utils.data import DataLoader, Dataset
import json
import os
import pandas as pd
from PIL import Image
import torchvision.transforms as transforms
from tqdm import tqdm
from ResNet_LSTM_Model2 import ResNetFeatureExtractor, VideoClassifier  # (import your models)
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import subprocess
from glob import glob
import random

In [9]:
#0. Convert Videos to Frames
# Constants
SOURCE_VIDEO_DIR = "Test_Video"
OUTPUT_FRAMES_DIR = "Test_Video_Frames"
os.makedirs(OUTPUT_FRAMES_DIR, exist_ok=True)

# Gather all .mp4 files from both Normal and Crash folders
video_files = glob(os.path.join(SOURCE_VIDEO_DIR, "Normal-002", "*.mp4")) + \
              glob(os.path.join(SOURCE_VIDEO_DIR, "Crash-1500", "*.mp4"))

print(f"Found {len(video_files)} videos for testing.")

# Function to extract frames
def videos_to_frames(video_path, output_dir, fps=10):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    # load module first inside the shell!
    query = f"module load ffmpeg && ffmpeg -i '{video_path}' -vf fps={fps} '{os.path.join(output_dir, '%06d.jpg')}' -hide_banner -loglevel error"
    print("Running:", query)
    
    result = subprocess.run(query, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

    if result.returncode != 0:
        print("ffmpeg failed:")
        print(result.stderr.decode())
    else:
        print("ffmpeg succeeded.")

    return len([f for f in os.listdir(output_dir) if f.endswith(".jpg")])

# Process all videos
for video_path in video_files:
    video_name = os.path.splitext(os.path.basename(video_path))[0]
    output_dir = os.path.join(OUTPUT_FRAMES_DIR, video_name)
    videos_to_frames(video_path, output_dir, fps=10)

Found 4500 videos for testing.
Running: module load ffmpeg && ffmpeg -i 'Test_Video/Normal-002/normal_001102.mp4' -vf fps=10 'Test_Video_Frames/normal_001102/%06d.jpg' -hide_banner -loglevel error
ffmpeg succeeded.
Running: module load ffmpeg && ffmpeg -i 'Test_Video/Normal-002/normal_001724.mp4' -vf fps=10 'Test_Video_Frames/normal_001724/%06d.jpg' -hide_banner -loglevel error
ffmpeg succeeded.
Running: module load ffmpeg && ffmpeg -i 'Test_Video/Normal-002/normal_001937.mp4' -vf fps=10 'Test_Video_Frames/normal_001937/%06d.jpg' -hide_banner -loglevel error
ffmpeg succeeded.
Running: module load ffmpeg && ffmpeg -i 'Test_Video/Normal-002/normal_001910.mp4' -vf fps=10 'Test_Video_Frames/normal_001910/%06d.jpg' -hide_banner -loglevel error
ffmpeg succeeded.
Running: module load ffmpeg && ffmpeg -i 'Test_Video/Normal-002/normal_002467.mp4' -vf fps=10 'Test_Video_Frames/normal_002467/%06d.jpg' -hide_banner -loglevel error
ffmpeg succeeded.
Running: module load ffmpeg && ffmpeg -i 'Test_Vi

ffmpeg succeeded.
Running: module load ffmpeg && ffmpeg -i 'Test_Video/Normal-002/normal_000972.mp4' -vf fps=10 'Test_Video_Frames/normal_000972/%06d.jpg' -hide_banner -loglevel error
ffmpeg succeeded.
Running: module load ffmpeg && ffmpeg -i 'Test_Video/Normal-002/normal_001942.mp4' -vf fps=10 'Test_Video_Frames/normal_001942/%06d.jpg' -hide_banner -loglevel error
ffmpeg succeeded.
Running: module load ffmpeg && ffmpeg -i 'Test_Video/Normal-002/normal_000418.mp4' -vf fps=10 'Test_Video_Frames/normal_000418/%06d.jpg' -hide_banner -loglevel error
ffmpeg succeeded.
Running: module load ffmpeg && ffmpeg -i 'Test_Video/Normal-002/normal_000729.mp4' -vf fps=10 'Test_Video_Frames/normal_000729/%06d.jpg' -hide_banner -loglevel error
ffmpeg succeeded.
Running: module load ffmpeg && ffmpeg -i 'Test_Video/Normal-002/normal_002326.mp4' -vf fps=10 'Test_Video_Frames/normal_002326/%06d.jpg' -hide_banner -loglevel error
ffmpeg succeeded.
Running: module load ffmpeg && ffmpeg -i 'Test_Video/Normal-00

ffmpeg succeeded.
Running: module load ffmpeg && ffmpeg -i 'Test_Video/Normal-002/normal_000492.mp4' -vf fps=10 'Test_Video_Frames/normal_000492/%06d.jpg' -hide_banner -loglevel error
ffmpeg succeeded.
Running: module load ffmpeg && ffmpeg -i 'Test_Video/Normal-002/normal_002826.mp4' -vf fps=10 'Test_Video_Frames/normal_002826/%06d.jpg' -hide_banner -loglevel error
ffmpeg succeeded.
Running: module load ffmpeg && ffmpeg -i 'Test_Video/Normal-002/normal_002955.mp4' -vf fps=10 'Test_Video_Frames/normal_002955/%06d.jpg' -hide_banner -loglevel error
ffmpeg succeeded.
Running: module load ffmpeg && ffmpeg -i 'Test_Video/Normal-002/normal_002678.mp4' -vf fps=10 'Test_Video_Frames/normal_002678/%06d.jpg' -hide_banner -loglevel error
ffmpeg succeeded.
Running: module load ffmpeg && ffmpeg -i 'Test_Video/Normal-002/normal_001851.mp4' -vf fps=10 'Test_Video_Frames/normal_001851/%06d.jpg' -hide_banner -loglevel error
ffmpeg succeeded.
Running: module load ffmpeg && ffmpeg -i 'Test_Video/Normal-00

ffmpeg succeeded.
Running: module load ffmpeg && ffmpeg -i 'Test_Video/Normal-002/normal_001673.mp4' -vf fps=10 'Test_Video_Frames/normal_001673/%06d.jpg' -hide_banner -loglevel error
ffmpeg succeeded.
Running: module load ffmpeg && ffmpeg -i 'Test_Video/Normal-002/normal_000931.mp4' -vf fps=10 'Test_Video_Frames/normal_000931/%06d.jpg' -hide_banner -loglevel error
ffmpeg succeeded.
Running: module load ffmpeg && ffmpeg -i 'Test_Video/Normal-002/normal_001887.mp4' -vf fps=10 'Test_Video_Frames/normal_001887/%06d.jpg' -hide_banner -loglevel error
ffmpeg succeeded.
Running: module load ffmpeg && ffmpeg -i 'Test_Video/Normal-002/normal_000183.mp4' -vf fps=10 'Test_Video_Frames/normal_000183/%06d.jpg' -hide_banner -loglevel error
ffmpeg succeeded.
Running: module load ffmpeg && ffmpeg -i 'Test_Video/Normal-002/normal_000503.mp4' -vf fps=10 'Test_Video_Frames/normal_000503/%06d.jpg' -hide_banner -loglevel error
ffmpeg succeeded.
Running: module load ffmpeg && ffmpeg -i 'Test_Video/Normal-00

ffmpeg succeeded.
Running: module load ffmpeg && ffmpeg -i 'Test_Video/Normal-002/normal_001690.mp4' -vf fps=10 'Test_Video_Frames/normal_001690/%06d.jpg' -hide_banner -loglevel error
ffmpeg succeeded.
Running: module load ffmpeg && ffmpeg -i 'Test_Video/Normal-002/normal_002068.mp4' -vf fps=10 'Test_Video_Frames/normal_002068/%06d.jpg' -hide_banner -loglevel error
ffmpeg succeeded.
Running: module load ffmpeg && ffmpeg -i 'Test_Video/Normal-002/normal_000499.mp4' -vf fps=10 'Test_Video_Frames/normal_000499/%06d.jpg' -hide_banner -loglevel error
ffmpeg succeeded.
Running: module load ffmpeg && ffmpeg -i 'Test_Video/Normal-002/normal_000143.mp4' -vf fps=10 'Test_Video_Frames/normal_000143/%06d.jpg' -hide_banner -loglevel error
ffmpeg succeeded.
Running: module load ffmpeg && ffmpeg -i 'Test_Video/Normal-002/normal_000465.mp4' -vf fps=10 'Test_Video_Frames/normal_000465/%06d.jpg' -hide_banner -loglevel error
ffmpeg succeeded.
Running: module load ffmpeg && ffmpeg -i 'Test_Video/Normal-00

...

ffmpeg succeeded.
Running: module load ffmpeg && ffmpeg -i 'Test_Video/Crash-1500/crash_001118.mp4' -vf fps=10 'Test_Video_Frames/crash_001118/%06d.jpg' -hide_banner -loglevel error
ffmpeg succeeded.
Running: module load ffmpeg && ffmpeg -i 'Test_Video/Crash-1500/crash_000731.mp4' -vf fps=10 'Test_Video_Frames/crash_000731/%06d.jpg' -hide_banner -loglevel error
ffmpeg succeeded.
Running: module load ffmpeg && ffmpeg -i 'Test_Video/Crash-1500/crash_000633.mp4' -vf fps=10 'Test_Video_Frames/crash_000633/%06d.jpg' -hide_banner -loglevel error
ffmpeg succeeded.
Running: module load ffmpeg && ffmpeg -i 'Test_Video/Crash-1500/crash_000737.mp4' -vf fps=10 'Test_Video_Frames/crash_000737/%06d.jpg' -hide_banner -loglevel error
ffmpeg succeeded.
Running: module load ffmpeg && ffmpeg -i 'Test_Video/Crash-1500/crash_000552.mp4' -vf fps=10 'Test_Video_Frames/crash_000552/%06d.jpg' -hide_banner -loglevel error
ffmpeg succeeded.
Running: module load ffmpeg && ffmpeg -i 'Test_Video/Crash-1500/crash_00

ffmpeg succeeded.
Running: module load ffmpeg && ffmpeg -i 'Test_Video/Crash-1500/crash_000134.mp4' -vf fps=10 'Test_Video_Frames/crash_000134/%06d.jpg' -hide_banner -loglevel error
ffmpeg succeeded.
Running: module load ffmpeg && ffmpeg -i 'Test_Video/Crash-1500/crash_000975.mp4' -vf fps=10 'Test_Video_Frames/crash_000975/%06d.jpg' -hide_banner -loglevel error
ffmpeg succeeded.
Running: module load ffmpeg && ffmpeg -i 'Test_Video/Crash-1500/crash_000152.mp4' -vf fps=10 'Test_Video_Frames/crash_000152/%06d.jpg' -hide_banner -loglevel error
ffmpeg succeeded.
Running: module load ffmpeg && ffmpeg -i 'Test_Video/Crash-1500/crash_001413.mp4' -vf fps=10 'Test_Video_Frames/crash_001413/%06d.jpg' -hide_banner -loglevel error
ffmpeg succeeded.
Running: module load ffmpeg && ffmpeg -i 'Test_Video/Crash-1500/crash_000319.mp4' -vf fps=10 'Test_Video_Frames/crash_000319/%06d.jpg' -hide_banner -loglevel error
ffmpeg succeeded.
Running: module load ffmpeg && ffmpeg -i 'Test_Video/Crash-1500/crash_00

ffmpeg succeeded.
Running: module load ffmpeg && ffmpeg -i 'Test_Video/Crash-1500/crash_001313.mp4' -vf fps=10 'Test_Video_Frames/crash_001313/%06d.jpg' -hide_banner -loglevel error
ffmpeg succeeded.
Running: module load ffmpeg && ffmpeg -i 'Test_Video/Crash-1500/crash_001048.mp4' -vf fps=10 'Test_Video_Frames/crash_001048/%06d.jpg' -hide_banner -loglevel error
ffmpeg succeeded.
Running: module load ffmpeg && ffmpeg -i 'Test_Video/Crash-1500/crash_000848.mp4' -vf fps=10 'Test_Video_Frames/crash_000848/%06d.jpg' -hide_banner -loglevel error
ffmpeg succeeded.
Running: module load ffmpeg && ffmpeg -i 'Test_Video/Crash-1500/crash_001134.mp4' -vf fps=10 'Test_Video_Frames/crash_001134/%06d.jpg' -hide_banner -loglevel error
ffmpeg succeeded.
Running: module load ffmpeg && ffmpeg -i 'Test_Video/Crash-1500/crash_001493.mp4' -vf fps=10 'Test_Video_Frames/crash_001493/%06d.jpg' -hide_banner -loglevel error
ffmpeg succeeded.
Running: module load ffmpeg && ffmpeg -i 'Test_Video/Crash-1500/crash_00

ffmpeg succeeded.
Running: module load ffmpeg && ffmpeg -i 'Test_Video/Crash-1500/crash_000448.mp4' -vf fps=10 'Test_Video_Frames/crash_000448/%06d.jpg' -hide_banner -loglevel error
ffmpeg succeeded.
Running: module load ffmpeg && ffmpeg -i 'Test_Video/Crash-1500/crash_000025.mp4' -vf fps=10 'Test_Video_Frames/crash_000025/%06d.jpg' -hide_banner -loglevel error
ffmpeg succeeded.
Running: module load ffmpeg && ffmpeg -i 'Test_Video/Crash-1500/crash_000727.mp4' -vf fps=10 'Test_Video_Frames/crash_000727/%06d.jpg' -hide_banner -loglevel error
ffmpeg succeeded.
Running: module load ffmpeg && ffmpeg -i 'Test_Video/Crash-1500/crash_000531.mp4' -vf fps=10 'Test_Video_Frames/crash_000531/%06d.jpg' -hide_banner -loglevel error
ffmpeg succeeded.
Running: module load ffmpeg && ffmpeg -i 'Test_Video/Crash-1500/crash_000645.mp4' -vf fps=10 'Test_Video_Frames/crash_000645/%06d.jpg' -hide_banner -loglevel error
ffmpeg succeeded.
Running: module load ffmpeg && ffmpeg -i 'Test_Video/Crash-1500/crash_00

ffmpeg succeeded.
Running: module load ffmpeg && ffmpeg -i 'Test_Video/Crash-1500/crash_000845.mp4' -vf fps=10 'Test_Video_Frames/crash_000845/%06d.jpg' -hide_banner -loglevel error
ffmpeg succeeded.
Running: module load ffmpeg && ffmpeg -i 'Test_Video/Crash-1500/crash_000949.mp4' -vf fps=10 'Test_Video_Frames/crash_000949/%06d.jpg' -hide_banner -loglevel error
ffmpeg succeeded.
Running: module load ffmpeg && ffmpeg -i 'Test_Video/Crash-1500/crash_000251.mp4' -vf fps=10 'Test_Video_Frames/crash_000251/%06d.jpg' -hide_banner -loglevel error
ffmpeg succeeded.
Running: module load ffmpeg && ffmpeg -i 'Test_Video/Crash-1500/crash_000865.mp4' -vf fps=10 'Test_Video_Frames/crash_000865/%06d.jpg' -hide_banner -loglevel error
ffmpeg succeeded.
Running: module load ffmpeg && ffmpeg -i 'Test_Video/Crash-1500/crash_000132.mp4' -vf fps=10 'Test_Video_Frames/crash_000132/%06d.jpg' -hide_banner -loglevel error
ffmpeg succeeded.
Running: module load ffmpeg && ffmpeg -i 'Test_Video/Crash-1500/crash_00

In [18]:
from torchvision.transforms.autoaugment import AutoAugment, AutoAugmentPolicy
import numpy as np

# 1. Define the Dataset
class TestVideoDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform, sample_count=64):
        self.data = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform
        self.sample_count = sample_count
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        video_info = self.data.iloc[idx]
        video_folder = os.path.join(self.root_dir, os.path.splitext(os.path.basename(video_info['video_path']))[0])

        frame_paths = sorted([os.path.join(video_folder, f) for f in os.listdir(video_folder) if f.endswith('.jpg')])

        total_frames = len(frame_paths)
        if total_frames == 0:
            raise ValueError(f"No frames found for {video_folder}")

        step = max(total_frames // self.sample_count, 1)
        selected_frames = [frame_paths[min(i * step, total_frames - 1)] for i in range(self.sample_count)]
        frames = [self.transform(Image.open(f).convert('RGB')) for f in selected_frames]
        
        label = video_info['label']
        return torch.stack(frames), torch.tensor(label, dtype=torch.float32)

# 2. Transform (same as training)
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.RandomPerspective(distortion_scale=0.2, p=0.5),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.GaussianBlur(kernel_size=(3, 3), sigma=(0.1, 2.0)),
    AutoAugment(policy=AutoAugmentPolicy.IMAGENET),
    transforms.ToTensor(),
    transforms.RandomErasing(p=0.5),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                       std=[0.229, 0.224, 0.225])
])

# 3. Load the Dataset
test_dataset = TestVideoDataset(
    csv_file="test_labels.csv",              # You need to create this earlier (video names + labels)
    root_dir="Test_Video_Frames",             # <--- notice here: it's the FRAMES folder
    transform=transform
)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False, num_workers=4)

# 4. Load your Best Model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

feature_extractor = ResNetFeatureExtractor().to(device)
classifier = VideoClassifier(feature_dim=feature_extractor.output_dim).to(device)

checkpoint = torch.load("best_model.pth", map_location=device, weights_only=False)
classifier.load_state_dict(checkpoint['model_state_dict'])

classifier.eval()
feature_extractor.eval()

# 5. Predict on Test Set (modified to collect raw outputs)
all_outputs, all_labels = [], []

with torch.no_grad():
    for frames, labels in tqdm(test_loader):
        frames = frames.to(device)  # [batch, seq, C, H, W]
        labels = labels.to(device)

        batch_size, seq_len, C, H, W = frames.shape
        frames = frames.view(batch_size * seq_len, C, H, W)
        features = feature_extractor(frames)
        features = features.view(batch_size, seq_len, -1)

        outputs = classifier(features)

        all_outputs.extend(outputs.squeeze().cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

all_outputs = np.array(all_outputs)
all_labels = np.array(all_labels)

# 6. Threshold Tuning
thresholds = np.arange(0.1, 0.91, 0.05)  # From 0.1 to 0.9 with step 0.05

best_f1 = 0
best_threshold = 0

print("\nThreshold Tuning Results:")
print("Threshold | Accuracy | Precision | Recall | F1-Score")
print("-" * 50)

for threshold in thresholds:
    preds = (all_outputs > threshold).astype(int)

    acc = accuracy_score(all_labels, preds)
    prec = precision_score(all_labels, preds, zero_division=0)
    rec = recall_score(all_labels, preds, zero_division=0)
    f1 = f1_score(all_labels, preds, zero_division=0)

    print(f"{threshold:.2f}      | {acc:.4f} | {prec:.4f} | {rec:.4f} | {f1:.4f}")

    if f1 > best_f1:
        best_f1 = f1
        best_threshold = threshold

print("\nBest Threshold:", best_threshold)
print("Best F1-Score:", best_f1)

/usr4/dl4ds/nikkili/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr4/dl4ds/nikkili/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
100%|██████████| 563/563 [14:51<00:00,  1.58s/it]



Threshold Tuning Results:
Threshold | Accuracy | Precision | Recall | F1-Score
--------------------------------------------------
0.10      | 0.8829 | 0.9542 | 0.6813 | 0.7950
0.15      | 0.8627 | 0.9594 | 0.6140 | 0.7488
0.20      | 0.8458 | 0.9643 | 0.5580 | 0.7069
0.25      | 0.8302 | 0.9670 | 0.5080 | 0.6661
0.30      | 0.8178 | 0.9709 | 0.4673 | 0.6310
0.35      | 0.8071 | 0.9773 | 0.4313 | 0.5985
0.40      | 0.7964 | 0.9787 | 0.3980 | 0.5659
0.45      | 0.7856 | 0.9785 | 0.3647 | 0.5313
0.50      | 0.7764 | 0.9805 | 0.3360 | 0.5005
0.55      | 0.7660 | 0.9806 | 0.3040 | 0.4641
0.60      | 0.7549 | 0.9806 | 0.2700 | 0.4234
0.65      | 0.7467 | 0.9865 | 0.2433 | 0.3904
0.70      | 0.7376 | 0.9848 | 0.2160 | 0.3543
0.75      | 0.7282 | 0.9860 | 0.1873 | 0.3148
0.80      | 0.7191 | 0.9836 | 0.1600 | 0.2752
0.85      | 0.7096 | 0.9949 | 0.1293 | 0.2289
0.90      | 0.6971 | 1.0000 | 0.0913 | 0.1674

Best Threshold: 0.1
Best F1-Score: 0.7950213924542979
